# Entra Agent ID + Conditional Access for Hosted Agents

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **© 2025 Ozgur Guler. All rights reserved.**

---

This notebook demonstrates how to:
1. Find your hosted agent's identity in Microsoft Entra
2. Create a Conditional Access policy targeting the agent
3. Test and verify policy enforcement

## Prerequisites

- Completed `../02-azd-deploy-hosted-agent` (working hosted agent)
- Microsoft Entra ID P1 or P2 license (for Conditional Access)
- Conditional Access Administrator role in your tenant
- Azure CLI installed and authenticated

---

## What is Microsoft Entra Agent ID?

**Microsoft Entra Agent ID** is a specialized identity framework for AI agents. It provides:

| Feature | Description |
|---------|-------------|
| **Agent Identity** | Unique, first-class identity for each agent (like user identities) |
| **Conditional Access** | Apply access policies to agents based on risk, location, etc. |
| **Identity Protection** | Detect and block risky agent behavior |
| **Governance** | Lifecycle management, owners, sponsors, expiration |

### How Foundry Creates Agent Identities

When you deploy a hosted agent:

1. **Unpublished agents** share the project's managed identity
2. **Published agents** get a dedicated Agent Identity automatically

The agent identity appears in:
- Azure Portal (JSON View of your agent resource)
- Microsoft Entra Admin Center (Agent ID tab)

---

## Section 1: Configuration

In [ ]:
import os
from dotenv import load_dotenv

# Load environment from parent directory
load_dotenv("../.env")

# Configuration
SUBSCRIPTION_ID = os.getenv("AZ_SUBSCRIPTION_ID", "a20bc194-9787-44ee-9c7f-7c3130e651b6")
RESOURCE_GROUP = os.getenv("AZ_RESOURCE_GROUP", "rg-ozgurguler-7212")
FOUNDRY_ACCOUNT = os.getenv("FOUNDRY_ACCOUNT_NAME", "ozgurguler-7212-resource")
PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME", "ozgurguler-7212")
HOSTED_AGENT_NAME = "my-hosted-agent"

print(f"Subscription: {SUBSCRIPTION_ID}")
print(f"Resource Group: {RESOURCE_GROUP}")
print(f"Foundry Account: {FOUNDRY_ACCOUNT}")
print(f"Project: {PROJECT_NAME}")
print(f"Hosted Agent: {HOSTED_AGENT_NAME}")

---

## Section 2: Find the Agent Identity

Our hosted agent uses the **project's managed identity** for authentication. Let's find it.

In [ ]:
# Get the Foundry project's identity (shared by all agents in the project)
import subprocess
import json

# Get the Cognitive Services account (Foundry account) details
result = subprocess.run(
    ["az", "cognitiveservices", "account", "show",
     "--name", FOUNDRY_ACCOUNT,
     "--resource-group", RESOURCE_GROUP,
     "-o", "json"],
    capture_output=True, text=True
)

account_info = json.loads(result.stdout)
print("Foundry Account Identity:")
print(json.dumps(account_info.get("identity", {}), indent=2))

In [ ]:
# Extract the principal ID - this is what we'll target with Conditional Access
identity = account_info.get("identity", {})

AGENT_PRINCIPAL_ID = identity.get("principalId")
AGENT_TENANT_ID = identity.get("tenantId")

print(f"Agent Principal ID: {AGENT_PRINCIPAL_ID}")
print(f"Tenant ID: {AGENT_TENANT_ID}")
print()
print("This is the identity your hosted agent uses to authenticate.")
print("Conditional Access policies targeting this identity will affect your agent.")

### View in Entra Admin Center

You can also view agent identities in the Microsoft Entra Admin Center:

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Entra ID > Agent ID > Agent identities**
3. Look for your agent in the list

Or view the service principal directly:

In [ ]:
# View the service principal in Entra
if AGENT_PRINCIPAL_ID:
    result = subprocess.run(
        ["az", "ad", "sp", "show", "--id", AGENT_PRINCIPAL_ID, "-o", "json"],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        sp_info = json.loads(result.stdout)
        print("Service Principal Details:")
        print(f"  Display Name: {sp_info.get('displayName')}")
        print(f"  App ID: {sp_info.get('appId')}")
        print(f"  Object ID: {sp_info.get('id')}")
        print(f"  Type: {sp_info.get('servicePrincipalType')}")
    else:
        print(f"Note: Could not retrieve SP details. Error: {result.stderr}")
        print("The managed identity may not be visible as a standard service principal.")

---

## Section 3: Create a Conditional Access Policy

### The Simplest Demo: Block Agent Access to a Specific Resource

We'll create a Conditional Access policy that:
1. Targets our agent identity
2. Blocks access to Azure OpenAI (the resource our agent uses)
3. Shows enforcement in sign-in logs

**Policy Logic:**
```
IF agent_identity = our_hosted_agent
AND target_resource = Azure Cognitive Services
THEN BLOCK
```

### Important: Creating CA Policies Requires Portal or Graph API

Conditional Access policies for agents are created via:
- **Microsoft Entra Admin Center** (recommended for first-time setup)
- **Microsoft Graph API** (for automation)

### Option A: Create Policy via Entra Admin Center (Manual Steps)

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Protection > Conditional Access > Policies**
3. Click **+ New policy**
4. Configure:

| Setting | Value |
|---------|-------|
| **Name** | `Block Hosted Agent - Demo` |
| **Assignments > Users, agents, or workload identities** | Select **Agents (Preview)** |
| **Include** | Select agent identities > Choose your agent's identity |
| **Target resources** | All resources (or select Azure Cognitive Services) |
| **Grant** | Block access |
| **Enable policy** | Report-only (for testing) or On |

5. Click **Create**

### Option B: Create Policy via Microsoft Graph API

For programmatic creation, use the Microsoft Graph API. First, get an access token:

In [ ]:
# Get access token for Microsoft Graph
result = subprocess.run(
    ["az", "account", "get-access-token", 
     "--resource", "https://graph.microsoft.com",
     "-o", "json"],
    capture_output=True, text=True
)

if result.returncode == 0:
    token_info = json.loads(result.stdout)
    GRAPH_TOKEN = token_info.get("accessToken")
    print("Got Microsoft Graph access token")
    print(f"Expires: {token_info.get('expiresOn')}")
else:
    print(f"Error getting token: {result.stderr}")
    GRAPH_TOKEN = None

In [ ]:
# List existing Conditional Access policies
import requests

if GRAPH_TOKEN:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    resp = requests.get(
        "https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies",
        headers=headers
    )
    
    if resp.status_code == 200:
        policies = resp.json().get("value", [])
        print(f"Found {len(policies)} existing Conditional Access policies:")
        for p in policies[:10]:  # Show first 10
            state = p.get('state', 'unknown')
            print(f"  - {p.get('displayName')} ({state})")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
        print("\nNote: You need Conditional Access Administrator role to list policies.")

In [ ]:
# Create a Conditional Access policy targeting the agent identity
# This policy will BLOCK the agent from accessing Azure Cognitive Services

POLICY_NAME = "Block Hosted Agent - Demo"

# Azure Cognitive Services app ID (this is what Azure OpenAI uses)
COGNITIVE_SERVICES_APP_ID = "fcc0e4a3-794f-4b44-a6a2-bf75d57a9644"  # Azure Cognitive Services

policy_body = {
    "displayName": POLICY_NAME,
    "state": "enabledForReportingButNotEnforced",  # Report-only mode for testing
    "conditions": {
        "clientApplications": {
            "includeServicePrincipals": [AGENT_PRINCIPAL_ID] if AGENT_PRINCIPAL_ID else [],
            "excludeServicePrincipals": []
        },
        "applications": {
            "includeApplications": [COGNITIVE_SERVICES_APP_ID],
            "excludeApplications": []
        }
    },
    "grantControls": {
        "operator": "OR",
        "builtInControls": ["block"]
    }
}

print("Policy configuration:")
print(json.dumps(policy_body, indent=2))

In [ ]:
# Actually create the policy (uncomment to execute)
# WARNING: This will create a real Conditional Access policy!

CREATE_POLICY = False  # Set to True to create the policy

if CREATE_POLICY and GRAPH_TOKEN and AGENT_PRINCIPAL_ID:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    resp = requests.post(
        "https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies",
        headers=headers,
        json=policy_body
    )
    
    if resp.status_code == 201:
        created_policy = resp.json()
        print(f"Policy created successfully!")
        print(f"  ID: {created_policy.get('id')}")
        print(f"  Name: {created_policy.get('displayName')}")
        print(f"  State: {created_policy.get('state')}")
    else:
        print(f"Error creating policy: {resp.status_code}")
        print(resp.text)
else:
    print("Policy creation skipped (CREATE_POLICY = False)")
    print("\nTo create the policy:")
    print("1. Set CREATE_POLICY = True above")
    print("2. Re-run this cell")
    print("\nOr create manually in the Entra Admin Center (see Option A above)")

---

## Section 4: Test Policy Enforcement

Now let's test the policy by invoking our hosted agent.

In [ ]:
# First, let's invoke the agent WITHOUT the blocking policy (baseline)
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

PROJECT_ENDPOINT = f"https://{FOUNDRY_ACCOUNT}.services.ai.azure.com/api/projects/{PROJECT_NAME}"

credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

print(f"Testing agent at: {PROJECT_ENDPOINT}")
print(f"Agent name: {HOSTED_AGENT_NAME}")

In [ ]:
# Invoke the agent
try:
    response = client.agents.invoke(
        agent_name=HOSTED_AGENT_NAME,
        content="Hello, what is the capital of France?"
    )
    
    print("Agent Response:")
    print("-" * 40)
    if hasattr(response, 'text'):
        print(response.text)
    elif hasattr(response, 'content'):
        print(response.content)
    else:
        print(response)
    print("-" * 40)
    print("\nAgent is working normally.")
    
except Exception as e:
    print(f"Error invoking agent: {e}")
    if "blocked" in str(e).lower() or "conditional access" in str(e).lower():
        print("\n>>> Conditional Access policy is BLOCKING the agent! <<<")

### Enable the Blocking Policy

Now enable the policy (change from Report-only to On):

**Via Entra Admin Center:**
1. Go to **Protection > Conditional Access > Policies**
2. Click on "Block Hosted Agent - Demo"
3. Change **Enable policy** from "Report-only" to **On**
4. Save

**Via Graph API:**

In [ ]:
# Update policy to enabled (blocking) state
# You need the policy ID from when it was created

POLICY_ID = ""  # Fill in your policy ID here
ENABLE_BLOCKING = False  # Set to True to enable blocking

if ENABLE_BLOCKING and POLICY_ID and GRAPH_TOKEN:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    # Change state to "enabled" (enforcing)
    resp = requests.patch(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers,
        json={"state": "enabled"}
    )
    
    if resp.status_code == 204:
        print("Policy enabled! Agent access is now BLOCKED.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy update skipped.")
    print("To enable blocking: set POLICY_ID and ENABLE_BLOCKING = True")

In [ ]:
# Test again - should now be blocked!
# Wait a minute or two for policy to propagate

import time
print("Waiting 30 seconds for policy propagation...")
time.sleep(30)

try:
    response = client.agents.invoke(
        agent_name=HOSTED_AGENT_NAME,
        content="Hello, what is 2 + 2?"
    )
    print("Agent Response:")
    print(response)
    print("\nNote: If you see a response, the policy may not be enforced yet.")
    
except Exception as e:
    error_msg = str(e).lower()
    if "blocked" in error_msg or "conditional access" in error_msg or "access denied" in error_msg:
        print(">>> SUCCESS: Conditional Access policy BLOCKED the agent! <<<")
        print(f"\nError details: {e}")
    else:
        print(f"Error (may be unrelated to CA): {e}")

---

## Section 5: View Sign-in Logs

Check the Entra sign-in logs to see Conditional Access evaluation.

### Via Entra Admin Center (Manual)

1. Go to [Microsoft Entra Admin Center](https://entra.microsoft.com)
2. Navigate to **Monitoring & health > Sign-in logs**
3. Filter by:
   - **Agent type**: Agent Identity or Agent ID user
   - **Is Agent**: Yes
4. Look for entries from your hosted agent
5. Click on an entry to see:
   - **Conditional Access tab**: Shows which policies were evaluated
   - **Result**: Success, Failure, or Blocked

In [ ]:
# Query sign-in logs via Graph API
# Note: Requires AuditLog.Read.All or Directory.Read.All permission

if GRAPH_TOKEN and AGENT_PRINCIPAL_ID:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    # Query service principal sign-ins for our agent
    filter_query = f"servicePrincipalId eq '{AGENT_PRINCIPAL_ID}'"
    
    resp = requests.get(
        f"https://graph.microsoft.com/v1.0/auditLogs/signIns",
        headers=headers,
        params={
            "$filter": filter_query,
            "$top": 10,
            "$orderby": "createdDateTime desc"
        }
    )
    
    if resp.status_code == 200:
        sign_ins = resp.json().get("value", [])
        print(f"Found {len(sign_ins)} recent sign-in events for the agent:")
        for si in sign_ins:
            print(f"\n  Time: {si.get('createdDateTime')}")
            print(f"  Status: {si.get('status', {}).get('errorCode', 'Success')}")
            print(f"  Resource: {si.get('resourceDisplayName')}")
            ca = si.get('conditionalAccessStatus', 'notApplied')
            print(f"  CA Status: {ca}")
    else:
        print(f"Error querying logs: {resp.status_code}")
        print("Note: You may need additional permissions (AuditLog.Read.All)")
else:
    print("Cannot query logs - missing token or principal ID")

---

## Section 6: Cleanup - Disable/Delete the Policy

After testing, disable or delete the blocking policy to restore agent functionality.

In [ ]:
# Disable the policy (set to report-only)
DISABLE_POLICY = False  # Set to True to disable

if DISABLE_POLICY and POLICY_ID and GRAPH_TOKEN:
    headers = {
        "Authorization": f"Bearer {GRAPH_TOKEN}",
        "Content-Type": "application/json"
    }
    
    resp = requests.patch(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers,
        json={"state": "enabledForReportingButNotEnforced"}  # Back to report-only
    )
    
    if resp.status_code == 204:
        print("Policy disabled (report-only). Agent access restored.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy disable skipped.")

In [ ]:
# Or delete the policy entirely
DELETE_POLICY = False  # Set to True to delete

if DELETE_POLICY and POLICY_ID and GRAPH_TOKEN:
    headers = {"Authorization": f"Bearer {GRAPH_TOKEN}"}
    
    resp = requests.delete(
        f"https://graph.microsoft.com/v1.0/identity/conditionalAccess/policies/{POLICY_ID}",
        headers=headers
    )
    
    if resp.status_code == 204:
        print("Policy deleted successfully.")
    else:
        print(f"Error: {resp.status_code} - {resp.text}")
else:
    print("Policy deletion skipped.")

---

## Summary

### What We Demonstrated

1. **Found the agent identity** - The managed identity of the Foundry project
2. **Created a Conditional Access policy** - Targeting the agent, blocking access to Cognitive Services
3. **Tested enforcement** - Agent calls fail when policy is enabled
4. **Viewed sign-in logs** - CA evaluation visible in Entra logs

### Key Takeaways

| Aspect | Finding |
|--------|--------|
| **Agent Identity** | Hosted agents use project's managed identity |
| **CA Policies** | Can target agents just like users |
| **Enforcement** | Block access to any Azure resource |
| **Visibility** | Sign-in logs show CA evaluation |

### Real-World Use Cases

1. **Block risky agents** - Automatically block agents with high risk scores
2. **Restrict resource access** - Only allow approved agents to access sensitive resources
3. **Compliance enforcement** - Ensure agents meet security requirements before accessing data
4. **Emergency kill switch** - Quickly disable compromised agents

---

## Next Steps

Continue to `../04-foundry-agent-memory` to add memory/state to your hosted agent.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>